# SGD Optimizer Utilizing Cyclic Learning Rate

Switch imports for backend

In [1]:
from keras.optimizers import *

#SWITCH FOR BACKEND

# import tensorflow as tensor
import theano.tensor as tensor

import tensorflow as tf
import numpy as np

Using Theano backend.
Using gpu device 1: GeForce GTX TITAN X (CNMeM is enabled with initial size: 80.0% of memory, cuDNN 5110)
/home/bckenstler/anaconda3/envs/py36/lib/python3.6/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


## Test CLR Function

Used following function to check if algo works correctly

In [102]:
def lr(iterations):
    """
    Test CLR function w/ following params:
    """
    step_size = 2000.
    lr = 0.01
    max_lr = 0.06
    cycle = np.floor(1+iterations/(2*step_size))
    x = np.abs(iterations/step_size - 2*cycle + 1)
    return lr + (max_lr-lr)*np.maximum(0, (1-x))

It should give the maximum lr of 0.06 at odd multiples of 2000, 0.01 at even multiples

In [120]:
lr(0)

0.01

In [121]:
lr(2000)

0.059999999999999998

In [122]:
lr(4000)

0.01

In [123]:
lr(6000)

0.059999999999999998

In [124]:
lr(8000)

0.01

In [125]:
lr(10000)

0.059999999999999998

Check

## Simple SGD Optimizer w/ CLR

### Optimizer

Keras sgd optimizer with triangular CLR

In [2]:
class SGD_CLR(Optimizer):
    """Basic Stochastic gradient descent optimizer.
    Supports triangular cyclic learning rate
    Includes attribute self.current_lr for testing purposes
    """

    def __init__(self, lr=0.01, max_lr=0.06, step_size=2000., **kwargs):
        super(SGD_CLR, self).__init__(**kwargs)
        self.iterations = K.variable(0., name='iterations')
        self.lr = K.variable(lr, name='lr')
        self.step_size = step_size
        self.max_lr = max_lr
        self.current_lr = K.variable(0., name='current_lr')

    def get_updates(self, params, constraints, loss):
        grads = self.get_gradients(loss, params)
        self.updates = [K.update_add(self.iterations, 1)]
        cycle = tensor.floor(1+self.iterations/(2*self.step_size))
        x = np.abs(self.iterations/self.step_size - 2*cycle + 1)
        lr = self.lr + (self.max_lr-self.lr)*K.maximum(0., (1-x))
        self.updates.append(K.update(self.current_lr, lr))
        # momentum
        shapes = [K.get_variable_shape(p) for p in params]
        moments = [K.zeros(shape) for shape in shapes]
        self.weights = [self.iterations] + moments
        for p, g in zip(params, grads):
            new_p = p - lr * g
            # apply constraints
            if p in constraints:
                c = constraints[p]
                new_p = c(new_p)

            self.updates.append(K.update(p, new_p))
        return self.updates

    def get_config(self):
        config = {'lr': float(K.get_value(self.lr)),
                  'momentum': float(K.get_value(self.momentum)),
                  'decay': float(K.get_value(self.decay)),
                  'nesterov': self.nesterov}
        base_config = super(SGD_CLR, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))



### Test Model

In [3]:
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Input

In [4]:
inp = Input(shape=(15,))                
x = Dense(10, activation='relu')(inp)
x = Dense(1, activation='sigmoid')(x)

model = Model(inp, x)

In [5]:
model.compile(optimizer=SGD_CLR(), loss = 'binary_crossentropy', metrics=['accuracy'])

In [6]:
X = np.random.rand(2000000,15)

In [7]:
Y = np.random.randint(0,2,size=2000000)

### THEANO Test

Set batch size so number of iterations in one epoch is CLR step_size (2000)

Current LR should be at max (0.06) for odd multiples of step_size(2000), and at base lr (0.01) for even multiples

In [8]:
for i in range(0,10):
    model.fit(X, Y, batch_size=1000, nb_epoch=1, verbose=0)
    print(f'Number of iterations: {model.optimizer.iterations.get_value()}')
    print(f'Current LR: {model.optimizer.current_lr.get_value()}')

Number of iterations: 2000.0
Current LR: 0.059974998235702515
Number of iterations: 4000.0
Current LR: 0.01002499833703041
Number of iterations: 6000.0
Current LR: 0.059974998235702515
Number of iterations: 8000.0
Current LR: 0.01002498622983694
Number of iterations: 10000.0
Current LR: 0.05997500941157341
Number of iterations: 12000.0
Current LR: 0.01002498622983694
Number of iterations: 14000.0
Current LR: 0.05997500941157341
Number of iterations: 16000.0
Current LR: 0.01002498622983694
Number of iterations: 18000.0
Current LR: 0.05997500941157341
Number of iterations: 20000.0
Current LR: 0.01002498622983694


Values check out

### Tensorflow Test

Unsure how to check values w/ Tensorflow variables